In [ ]:
! pip install transformers datasets evaluate
! pip install -U accelerate
! pip install -U transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# State-Pair classification

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Generate trimmed dataset and save it to drive

In [ ]:
import os
import csv
import json
import pandas as pd

base_path = '/content/drive/MyDrive/dataset_labeled/GroundTruthModels-SS'

file_ending = '.html.content_tags'
# file_ending = '.html.content'
# file_ending = '.html.tags'

In [ ]:
# # done only once in the beginning
# split = 'test_data'
# # split = 'train_data'

# def ensure_format(s):
#     if s.startswith(","): s = s[1:]
#     if not s.startswith('[\"'):
#         if not s.startswith('\"'): s = '[\"' + s
#         else: s = "[" + s

#     if not s.endswith('\"]'):
#         if not s.endswith('\"'): s = s + '\"]'
#         else: s = s + ']'

#     return s

# def trim_common_html(state1, state2):
#     """
#     Trims the common leading and trailing parts from two HTML page representations (content_tags, tags, content).

#     :param state1: HTML content of the first page as a string.
#     :param state2: HTML content of the second page as a string.
#     :return: A tuple of the trimmed HTML contents.
#     """
#     leading_common_length = 0
#     for x, y in zip(state1, state2):
#         if x == y:
#             leading_common_length += 1
#         else:
#             break
#     trailing_common_length = 0
#     for x, y in zip(reversed(state1[leading_common_length:]), reversed(state2[leading_common_length:])):
#         if x == y:
#             trailing_common_length += 1
#         else:
#             break
#     trimmed_state1 = state1[leading_common_length: len(state1) - trailing_common_length]
#     trimmed_state2 = state2[leading_common_length: len(state2) - trailing_common_length]
#     if trimmed_state1.startswith('\",'): trimmed_state1 = trimmed_state1[3:]
#     if trimmed_state2.startswith('\",'): trimmed_state2 = trimmed_state2[3:]

#     # if one page is subset of the other page
#     if trimmed_state2 == "" or trimmed_state1 == "":
#       return state1, state2

#     trimmed_states = [trimmed_state1, trimmed_state2]
#     corrected_states = [ensure_format(t) for t in trimmed_states]


#     trimmed_state1, trimmed_state2 = corrected_states


#     return trimmed_state1, trimmed_state2

# def fetch_file_content(application_name, state, file_ending):
#     file_path = os.path.join(base_path, application_name, state + file_ending)
#     try:
#       with open(file_path, 'r') as file:
#         return file.read()
#     except :
#       print(application_name, ", ", state, ".", file_ending)
#       return ""



# def chunk_html(html, chunk_size):
#     # if len(html) >= chunk_size: print("Chunking necessary")
#     return [html[i:i+chunk_size] for i in range(0, len(html), chunk_size)]

# print("GENERATE MODIFIED CSV FILE (INCLUDING THE TRIMMED STATES)")
# # saving the trimmed html in the dataset

# df = pd.read_csv(f'/content/drive/MyDrive/dataset_labeled/{split}.csv')
# def process_row(x):
#     content1 = fetch_file_content(x['appname'], x['state1'], file_ending)
#     content2 = fetch_file_content(x['appname'], x['state2'], file_ending)
#     trimmed1, trimmed2 = trim_common_html(content1, content2)
#     trimmed1, trimmed2 = content1, content2
#     try:
#         if trimmed1 == "":list1 = []
#         else: list1 = json.loads(trimmed1)
#         if trimmed2 == "":list2 = []
#         else: list2 = json.loads(trimmed2)
#     except json.JSONDecodeError:
#         list1, list2 = [], []
#         print("ERROR WHILE LOADING")
#         print(trimmed1)
#         print(trimmed2)
#         print("=")
#     joined1 = " ".join(list1) if len(list1) > 0 else "no content"
#     joined2 = " ".join(list2) if len(list2) > 0 else "no content"
#     return joined1, joined2

# df["trimmed_state1"], df["trimmed_state2"] = zip(*df.apply(process_row, axis=1))
# df.to_csv(f'/content/drive/MyDrive/dataset_labeled/modified_{split}_{file_ending.split(".")[-1]}.csv', index=False)

## Load dataset

Loading the dataset from 🤗:

In [ ]:
from datasets import load_dataset, Features, ClassLabel, Value, Dataset, DatasetDict

data_files = {"train": "modified_train_data_content_tags.csv", "test": "modified_test_data_content_tags.csv"}
dataset = load_dataset("lgk03/97k-state-pairs", data_files=data_files)

# transforming into binary classification
def map_labels(example):
    if example['HUMAN_CLASSIFICATION'] == 2:
        example['HUMAN_CLASSIFICATION'] = 0
    else:
        example['HUMAN_CLASSIFICATION'] = 1
    return example

dataset = dataset.map(map_labels)


## Preprocess

load a DistilBERT tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Preprocess:

In [ ]:
import torch
def preprocess(examples):
    tokenized_inputs = tokenizer(examples['trimmed_state1'], examples['trimmed_state2'],
                                 padding='max_length',
                                 truncation='longest_first',
                                 max_length=512,
                                 return_tensors='pt') # Return PyTorch tensors

    return {'input_ids': tokenized_inputs['input_ids'],
            'attention_mask': tokenized_inputs['attention_mask'],
            'labels': torch.tensor(examples['HUMAN_CLASSIFICATION'])
            }

In [ ]:
# tokenized_data = {split: ds.map(preprocess, batched=True, cache_file_name=None, batch_size=12) for split, ds in dataset.items()}

Map:   0%|          | 0/77870 [00:00<?, ? examples/s]

Map:   0%|          | 0/19471 [00:00<?, ? examples/s]

In [ ]:
# # SAVING THE TOKENIZED DS TO DRIVE FOR REUSE, in case execution stops
# for split, ds in tokenized_data.items():
#     # Define a path for each split in your Google Drive
#     drive_path = f"/content/drive/My Drive/tokenized_data/{split}"
#     ds.save_to_disk(drive_path)


Saving the dataset (0/1 shards):   0%|          | 0/77870 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/19471 [00:00<?, ? examples/s]

In [ ]:
# LOAD TOKENIZED DS FROM DRIVE
from datasets import load_from_disk
# vary name
print(file_ending)
tokenized_data = {split: load_from_disk(f"/content/drive/My Drive/tokenized_data_{file_ending.split('.')[-1]}_trimmed/{split}") for split in ['train', 'test']}


.html.content_tags


In [ ]:
tokenized_data

{'train': Dataset({
     features: ['appname', 'crawl', 'state1', 'state2', 'DOM_RTED', 'DOM_Levenshtein', 'DOM_contentHash', 'DOM_SIMHASH', 'VISUAL_BlockHash', 'VISUAL_PHash', 'VISUAL_Hyst', 'VISUAL_PDiff', 'VISUAL_SIFT', 'VISUAL_SSIM', 'HUMAN_CLASSIFICATION', 'TAGS', 'trimmed_state1', 'trimmed_state2', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 77870
 }),
 'test': Dataset({
     features: ['appname', 'crawl', 'state1', 'state2', 'DOM_RTED', 'DOM_Levenshtein', 'DOM_contentHash', 'DOM_SIMHASH', 'VISUAL_BlockHash', 'VISUAL_PHash', 'VISUAL_Hyst', 'VISUAL_PDiff', 'VISUAL_SIFT', 'VISUAL_SSIM', 'HUMAN_CLASSIFICATION', 'TAGS', 'trimmed_state1', 'trimmed_state2', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 19471
 })}

Create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the samples to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

Load F_1-Score as metric

In [ ]:
import evaluate

f1 = evaluate.load("f1")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## Train

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# num labels 2 => binary classification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1. Define training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments).
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune the model.

In [ ]:
training_args = TrainingArguments(
    output_dir=f"distilBERT-NDD{file_ending}",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub()